In [3]:
import polars as pl
df_inadim = pl.read_excel('sample/Inadimplencia Condomínio.xlsx')
print(df_inadim)

Could not determine dtype for column 0, falling back to string


shape: (560, 14)
┌────────────┬─────────────┬─────────────┬────────────┬───┬───────┬───────┬───────────┬────────────┐
│ Referência ┆ Condomínio  ┆ CNPJ        ┆ Nome do    ┆ … ┆ Multa ┆ Juros ┆ Vlr Total ┆ Status     │
│ ---        ┆ ---         ┆ ---         ┆ Pagador    ┆   ┆ ---   ┆ ---   ┆ ---       ┆ ---        │
│ str        ┆ str         ┆ str         ┆ ---        ┆   ┆ str   ┆ str   ┆ str       ┆ str        │
│            ┆             ┆             ┆ str        ┆   ┆       ┆       ┆           ┆            │
╞════════════╪═════════════╪═════════════╪════════════╪═══╪═══════╪═══════╪═══════════╪════════════╡
│ null       ┆ CONDOMINIO  ┆ 09.338.467/ ┆ ELIESDRAS  ┆ … ┆ 2,60  ┆ 4,98  ┆ 137,58    ┆ Vencido    │
│            ┆ RESIDENCIAL ┆ 0001-10     ┆ PATROCINIO ┆   ┆       ┆       ┆           ┆            │
│            ┆ SITIOCA…    ┆             ┆ SILVEIRA   ┆   ┆       ┆       ┆           ┆            │
│ null       ┆ CONDOMINIO  ┆ 09.338.467/ ┆ ELIESDRAS  ┆ … ┆ 2,60  ┆ 2,32  

In [51]:
df = df_inadim.select(
    pl.col('Nome do Pagador'),
    pl.col('Unidade').cast(pl.Int64),
    pl.col('Vlr Total').str.replace(',' , '.').cast(pl.Float64),
    pl.col('Status'),
)
print(df)

shape: (560, 4)
┌─────────────────────────────────┬─────────┬───────────┬────────────────────┐
│ Nome do Pagador                 ┆ Unidade ┆ Vlr Total ┆ Status             │
│ ---                             ┆ ---     ┆ ---       ┆ ---                │
│ str                             ┆ i64     ┆ f64       ┆ str                │
╞═════════════════════════════════╪═════════╪═══════════╪════════════════════╡
│ ELIESDRAS PATROCINIO SILVEIRA   ┆ 2       ┆ 137.58    ┆ Vencido            │
│ ELIESDRAS PATROCINIO SILVEIRA   ┆ 2       ┆ 134.92    ┆ Vencido            │
│ SIMONE NAYARA CASSIANO DA SILV… ┆ 4       ┆ 144.14    ┆ Vencido            │
│ SIMONE NAYARA CASSIANO DA SILV… ┆ 4       ┆ 142.72    ┆ Vencido            │
│ SIMONE NAYARA CASSIANO DA SILV… ┆ 4       ┆ 141.48    ┆ Vencido            │
│ …                               ┆ …       ┆ …         ┆ …                  │
│ EBANO AUGUSTO DE OLIVEIRA       ┆ 99      ┆ 140.19    ┆ Vencido   Advogado │
│ EBANO AUGUSTO DE OLIVEIRA       ┆ 

In [ ]:
df_grouped = df.group_by('Unidade', maintain_order=True).agg([
    pl.first('Nome do Pagador').alias('Proprietario'),
    pl.sum('Vlr Total').alias('Total devido')
])
print(df_grouped)

shape: (40, 3)
┌─────────┬─────────────────────────────────┬──────────────┐
│ Unidade ┆ Proprietario                    ┆ Total devido │
│ ---     ┆ ---                             ┆ ---          │
│ i64     ┆ str                             ┆ f64          │
╞═════════╪═════════════════════════════════╪══════════════╡
│ 2       ┆ ELIESDRAS PATROCINIO SILVEIRA   ┆ 272.5        │
│ 4       ┆ SIMONE NAYARA CASSIANO DA SILV… ┆ 1116.1       │
│ 7       ┆ MIRIAN GOMES MACHADO            ┆ 466.63       │
│ 9       ┆ MARIANA PEREIRAANUNCIACAO       ┆ 136.25       │
│ 12      ┆ TATIANA DA CRUZ TAVARES         ┆ 273.83       │
│ …       ┆ …                               ┆ …            │
│ 81      ┆ ANDERSON SILVA DA ANUNCIAÇÃO C… ┆ 953.06       │
│ 89      ┆ HORTENCIO MENDES DA SILVA       ┆ 5031.09      │
│ 93      ┆ JOCIMAR GUILHERME DA SLVA       ┆ 134.92       │
│ 94      ┆ AIR MARQUES MARTINS             ┆ 7087.31      │
│ 99      ┆ EBANO AUGUSTO DE OLIVEIRA       ┆ 9868.38      │
└────────